In [0]:
spark.conf.set(
  "fs.azure.account.key.snowflakesa2.blob.core.windows.net",
  "WbpRmcAjD57jy6vBIvB+wdv5rt9c5Nkb6BhUWPwySH90OhXvUxHvfQNTe8chnaWemX2Qo3RR37OT+AStgVvYWg=="
)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
schema = StructType([
    StructField("event_id", StringType(), True),
    StructField("event_time", StringType(), True),
    StructField("service_name", StringType(), True),
    StructField("log_level", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("api_endpoint", StringType(), True),
    StructField("response_time_ms", IntegerType(), True),
    StructField("error_message", StringType(), True),
    StructField("region", StringType(), True),
    StructField("_corrupt_record", StringType(), True)
])

In [0]:
df_raw = spark.read \
    .schema(schema) \
    .option("multiline", "true") \
    .json("wasbs://logs@snowflakesa2.blob.core.windows.net/app_logs.json")
df_raw.count()
df_raw.show(truncate=False)

+--------+--------------------+-------------+---------+-------+--------------+----------------+-------------+------+---------------+
|event_id|event_time          |service_name |log_level|user_id|api_endpoint  |response_time_ms|error_message|region|_corrupt_record|
+--------+--------------------+-------------+---------+-------+--------------+----------------+-------------+------+---------------+
|evt_101 |2025-09-10T10:15:30Z|order-service|ERROR    |U1001  |/api/v1/orders|850             |Timeout      |IN    |NULL           |
+--------+--------------------+-------------+---------+-------+--------------+----------------+-------------+------+---------------+



In [0]:
from pyspark.sql.functions import col, when, to_timestamp

In [0]:
df_clean = df_raw.select(
    col("event_id").alias("EVENT_ID"),
    to_timestamp(col("event_time")).alias("EVENT_TIME"),
    col("service_name").alias("SERVICE_NAME"),
    col("log_level").alias("LOG_LEVEL"),
    col("user_id").alias("USER_ID"),
    col("api_endpoint").alias("API_ENDPOINT"),
    col("response_time_ms").cast("int").alias("RESPONSE_TIME_MS"),
    col("error_message").alias("ERROR_MESSAGE"),
    col("region").alias("REGION"),
    when(col("log_level") == "ERROR", "Y").otherwise("N").alias("ERROR_FLAG")
)

df_clean.show()


+--------+-------------------+-------------+---------+-------+--------------+----------------+-------------+------+----------+
|EVENT_ID|         EVENT_TIME| SERVICE_NAME|LOG_LEVEL|USER_ID|  API_ENDPOINT|RESPONSE_TIME_MS|ERROR_MESSAGE|REGION|ERROR_FLAG|
+--------+-------------------+-------------+---------+-------+--------------+----------------+-------------+------+----------+
| evt_101|2025-09-10 10:15:30|order-service|    ERROR|  U1001|/api/v1/orders|             850|      Timeout|    IN|         Y|
+--------+-------------------+-------------+---------+-------+--------------+----------------+-------------+------+----------+



In [0]:
df_raw.write.mode("overwrite").format("delta").save("/tmp/app_logs_delta")

In [0]:
%pip install snowflake-snowpark-python

  Using cached snowflake_snowpark_python-1.44.0-py3-none-any.whl.metadata (16 kB)
  Using cached snowflake_connector_python-4.1.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (78 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached cffi-2.0.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of pyopenssl to determine which version is compatible with other requirements. This could take a while.
  Using cached pyopenssl-25.3.0-py3-none-any.whl.metadata (17 kB)
Using cached snowflake_snowpark_python-1.44.0-py3-none-any.whl (1.8 MB)
Using cached snowflake_connector_python-4.1.1-cp312-

In [0]:
from snowflake.snowpark import Session

In [0]:
connection_parameters = {
  "account": "QYQSSEQ-UT92541",
  "user": "BARATH46",
  "password": "Rbarathraj@2711",
  "role": "SYSADMIN",
  "warehouse": "LOGS_WH",
  "database": "LOG_ANALYTICS_DB",
  "schema": "PUBLIC"
}

session = Session.builder.configs(connection_parameters).create()

In [0]:
pdf = df_clean.toPandas()

In [0]:
session.write_pandas(
    pdf,
    table_name="APPLICATION_LOGS",
    overwrite=False
)